# Apache Hudi Core Conceptions (3) - MOR: File Layouts & File Sizing

*Author: [Laurence Geng](https://laurence.blog.csdn.net) @ [https://laurence.blog.csdn.net](https://laurence.blog.csdn.net)*

## 1. Configuration

In [1]:
%%sh
# deploy hudi bundle jar
hdfs dfs -copyFromLocal -f /usr/lib/hudi/hudi-spark-bundle.jar /tmp/hudi-spark-bundle.jar
hdfs dfs -ls /tmp/hudi-spark-bundle.jar

-rw-r--r--   1 emr-notebook hdfsadmingroup   61421977 2023-03-16 01:37 /tmp/hudi-spark-bundle.jar


In [2]:
%%configure -f
{
    "conf" : {
        "spark.jars":"hdfs:///tmp/hudi-spark-bundle.jar",            
        "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
        "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.hudi.catalog.HoodieCatalog"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
80,application_1678096020253_0135,spark,idle,Link,Link,None,
81,application_1678096020253_0140,spark,idle,Link,Link,None,


In [3]:
%env S3_BUCKET=apache-hudi-core-conceptions

env: S3_BUCKET=apache-hudi-core-conceptions


In [4]:
%%sql
set S3_BUCKET=apache-hudi-core-conceptions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
82,application_1678096020253_0141,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%env WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions

env: WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions


In [6]:
%%sh
# make workspace
mkdir -p $WORKSPACE
# deploy hudi-stat.sh, a utility shell script to output hudi table status
wget https://raw.githubusercontent.com/bluishglc/apache-hudi-core-conceptions/master/hudi-stat.sh -O $WORKSPACE/hudi-stat.sh &>/dev/null
chmod a+x $WORKSPACE/hudi-stat.sh
ls $WORKSPACE/hudi-stat.sh

/home/emr-notebook/apache-hudi-core-conceptions/hudi-stat.sh


In [7]:
%%html
<style>
table {float:left}
</style>

## 2. Test Case 1 - MOR: File Layouts + File Sizing

### 2.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Base File
2|Update|804KB|+1 Log File
3|Update|1.2MB|+1 Log File +1 Base File
4|Update|307MB|+1 Log File +1 Max Log File

### 2.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.compact.inline|false|true
hoodie.compact.schedule.inline|false|Default Value
hoodie.compact.inline.max.delta.commits|5|3
hoodie.logfile.max.size|1073741824 ( 1GB )|262144000 ( 250MB )
hoodie.copyonwrite.record.size.estimate|1024|175

### 2.3. Set Variables

In [8]:
%%sql
set TABLE_NAME=reviews_mor_layouts_sizing_1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%env TABLE_NAME=reviews_mor_layouts_sizing_1

env: TABLE_NAME=reviews_mor_layouts_sizing_1


### 2.4. Create Table

In [10]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [11]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.copyonwrite.record.size.estimate = '175',
    hoodie.compact.inline = 'true',
    -- hoodie.compact.schedule.inline = 'false',
    hoodie.compact.inline.max.delta.commits = '3',
    hoodie.logfile.max.size='262144000'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 2.5. Insert 96MB / +1 Base File

In [15]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316013805968 │ deltacommit │ COMPLETED │ 03-16 01:38 │ 03-16 01:38 │ 03-16 01:38 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 2.6. Update 803KB / +1 Log File

In [17]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [18]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316013805968 │ deltacommit │ COMPLETED │ 03-16 01:38 │ 03-16 01:38 │ 03-16 01:38 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316013930608 │ deltacommit │ COMPLETED │ 03-16 01:39 │ 03-16 01:39 │ 03-16 01:39 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 2.7. Update 1.2MB / +1 Log File +1 Base File

In [19]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '2003-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316013805968 │ deltacommit │ COMPLETED │ 03-16 01:38 │ 03-16 01:38 │ 03-16 01:38 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316013930608 │ deltacommit │ COMPLETED │ 03-16 01:39 │ 03-16 01:39 │ 03-16 01:39 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230316014013347 │ deltacommit │ COMPLETED │ 03-16 01:40 │ 03-16 01:40 │ 03-16 01:40 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230316014030291

### 2.8. Update 307MB / +1 Log File +1 Max Log File

In [21]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [22]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316013805968 │ deltacommit │ COMPLETED │ 03-16 01:38 │ 03-16 01:38 │ 03-16 01:38 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316013930608 │ deltacommit │ COMPLETED │ 03-16 01:39 │ 03-16 01:39 │ 03-16 01:39 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230316014013347 │ deltacommit │ COMPLETED │ 03-16 01:40 │ 03-16 01:40 │ 03-16 01:40 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230316014030291